# Configuracion general
Se configura el directorio en el donde se estara descargando la información, así como la query por la cual se guiará para buscar información

In [2]:
import os

# Directorio de los archivos
data_dirname = os.path.join(os.getcwd(), "data")

# Busqueda que se va a hacer
QUERY = "Reforma Judifical Mexico"

# Funcion para escribir un archivo de texto
def write_txt(text:str, name:str):
    output_file = os.path.join(data_dirname, f"{name}.txt")
    with open(output_file, "w", encoding="utf-8") as txt_file:
        txt_file.write(text)

# Recoleccion de datos por paginas web
Se utiliza BeautifulSoup para el scraping, asi como goolgesearch para hacer las busquedas, intenta encontrar el contenido más importante.

In [10]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search


# Funcion para buscar paginas en google
def search_pages_google(query: str):
    return list(search(query, num_results=30))


# Funcion para extraer el contenido orincipal de una pagina
def extract_page_content(url: str):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()

        main_content = (
            soup.find("main")
            or soup.find("article")
            or soup.find("div", class_="content")
            or soup.find("div", class_="main")
        )
        if main_content:
            return main_content.get_text(strip=True, separator="\n")
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None


# Buscar en las paginas
def search_in_pages():
    urls = search_pages_google(f"{QUERY} after:2024-01-01")
    text = ""
    for url in urls:
        content = extract_page_content(url)
        if content:
            text += f"---URL: {url}\n"
            text += content
            text += "\n\n"
    write_txt(text, "pages")


search_in_pages()

Error al procesar https://www.dof.gob.mx/nota_detalle.php?codigo=5738985&fecha=15/09/2024: HTTPSConnectionPool(host='www.dof.gob.mx', port=443): Max retries exceeded with url: /nota_detalle.php?codigo=5738985&fecha=15/09/2024 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al procesar https://www.cjf.gob.mx/reformaPJF/: HTTPSConnectionPool(host='www.cjf.gob.mx', port=443): Max retries exceeded with url: /reformaPJF/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al procesar https://law.stanford.edu/2024/11/05/la-reforma-judicial-en-mexico-viola-obligaciones-internacionales-en-materia-de-derechos-humanos/: 403 Client Error: Forbidden for url: https://law.stanford.edu/2024/11/05/la-reforma-judicial-en-mexico-viola-obligaciones-internacionales-

# Recolección de datos por pdf
Usando googlesearch en conjunto con requests, se extraen pdfs que sean despues del inicio de este año, para verificar que efectivamente sea esta reforma.

In [ ]:
import os
from typing import Dict
import requests
from googlesearch import search
from PyPDF2 import PdfReader

# Directorio de los pdfs
pdf_dirname = os.path.join(data_dirname, "pdfs")


# Funcion para descargar un pdf
def download_pdf(url: str):
    try:
        response = requests.get(url)
        response.raise_for_status()
        filename = url.split("/")[-1]
        if not filename.endswith(".pdf"):
            filename += ".pdf"
        os.makedirs(pdf_dirname, exist_ok=True)
        filepath = os.path.join(pdf_dirname, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)
        return filepath
    except Exception as e:
        print(f"Error al descargar {url}: {e}")


# Funcion para buscar pdfs
def search_pdfs() -> Dict[str, str]:
    results = search(f"{QUERY} filetype:pdf after:2024-01-01", num_results=30)
    results_dict = {}
    for url in results:
        if url.endswith(".pdf"):
            result = download_pdf(url)
            if result:
                results_dict[url] = result
    return results_dict


# Funcion para extraer texto de un pdf
def extract_text_from_pdf(filepath: str) -> str:
    try:
        text = ""
        with open(filepath, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                text += page.extract_text()
                text += "\n"
        return text
    except Exception as e:
        return ""


# Funcion para limpiar texto
def clean_text(text: str) -> str:
    lines = text.splitlines()
    cleaned_lines = [line.strip() for line in lines if line.strip()]
    return "\n".join(cleaned_lines)

# Funcion para crear un archivo de texto con los pdfs
def create_pdfs_txt():
    results = search_pdfs()
    text = ""
    for key, value in results.items():
        text += f"---URL: {key}\n"
        text += extract_text_from_pdf(value)
        text += "\n\n"

    write_txt(text, "pdfs")


create_pdfs_txt()

Error al descargar https://www.diputados.gob.mx/sedia/sia/spi/SAPI-ASS-11-24.pdf: HTTPSConnectionPool(host='www.diputados.gob.mx', port=443): Max retries exceeded with url: /sedia/sia/spi/SAPI-ASS-11-24.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al descargar https://escuelajudicial.cjf.gob.mx/MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf: HTTPSConnectionPool(host='escuelajudicial.cjf.gob.mx', port=443): Max retries exceeded with url: /MicroSitio/Doctos/Minuta_Reforma_Poder_Judicial.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Error al descargar https://www.cjf.gob.mx/reformapjf/documentos/Jornadas_Nacionales_Reforma_Poder_Judicial.pdf: HTTPSConnectionPool(host='www.cjf.gob.mx', port=443): Max retries exceeded with url: /reform

# Recoleccion de datos por videos
Por medio de pytube.fix se descargan los videos, se extrae el audio del video con ffmpeg y con openai-whisper se pasa a un archivo de txt.


In [ ]:
import whisper

# Cargar el modelo
model = whisper.load_model("base")

c:\Users\David\ia\.venv\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [22]:
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress
from pytubefix.contrib.search import Search, Filter
from pydub import AudioSegment

# Directorio de los audios
audios_dirname = os.path.join(data_dirname, "audios")

# Funcion para convertir un archivo a mp3
def convert_to_mp3(input_path: str, output_path: str) -> None:
    try:
        audio = AudioSegment.from_file(input_path)
        audio.export(output_path, format="mp3")
        print(f"Archivo convertido: {output_path}")
    except Exception as e:
        print(f"Error al convertir el archivo: {e}")

# Funcion para descargar los videos
def download_videos() -> Dict[str, str]:
    results = Search(
        query=QUERY,
        filters={"sort_by": Filter.get_sort_by("Relevance")},
    )
    i = 0
    results_dict = {}
    for result in results.videos:
        try:
            yt = YouTube(result.watch_url, on_progress_callback=on_progress)
            video = yt.streams.get_audio_only()
            video.download(output_path=audios_dirname, filename=f"{i}.m4a")
            results_dict[result.watch_url] = os.path.join(audios_dirname, f"{i}.m4a")
            i += 1
        except Exception as e:
            print(f"Error al descargar el video", e)

    return results_dict

# Funcion para transcribir un audio
def transcript_audio(audio_path: str) -> str:
    try:
        mp3_audio_path = audio_path.replace(".m4a", ".mp3")
        convert_to_mp3(audio_path, mp3_audio_path)
        result = model.transcribe(
            mp3_audio_path,
        )
        return result["text"]
    except Exception as e:
        print(f"Error al transcribir el audio: {e}")
        return None

# Funcion para buscar en los videos
def search_in_videos():
    results = download_videos()
    txt = ""
    for url, path in results.items():
        text = transcript_audio(path)
        if text:
            txt += f"---URL: {url}\n"
            txt += text
            txt += "\n\n"

    write_txt(txt, "audios")


search_in_videos()

Archivo convertido: c:\Users\David\ia\reforma\data\audios\0.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\1.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\2.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\3.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\4.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\5.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\6.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\7.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\8.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\9.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\10.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\11.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\12.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\13.mp3
Archivo convertido: c:\Users\David\ia\reforma\data\audios\14.mp3
Archivo convertido: c:\Users\David\

# Juntar todo en un mismo txt

In [4]:
import os

# Leer los txt
pages = open(os.path.join(data_dirname, "pages.txt"), "r", encoding="utf-8").read()
pdfs = open(os.path.join(data_dirname, "pdfs.txt"), "r", encoding="utf-8").read()
audios = open(os.path.join(data_dirname, "audios.txt"), "r", encoding="utf-8").read()

# Juntarlos en un mismo texto
all_text = pages + "\n\n" + pdfs
# + "\n\n" + audios

# Guardar el texto
write_txt(all_text, "all_text")